In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *


from PIL import Image
from numpy import array

import skimage
import time

import cv2
import numpy as np

from torchvision.utils import save_image
import torch

In [3]:
def segs2onehot(frame, nchannels):
    """
    Converts a tensor of shape (1, imgx, imgy)
    with discrete values in the range(nchannels)
    to a onehot style tensor of shape (nchannels, imgx, imgy)
    """
    *_, nx, ny = frame.shape
    x,y = np.meshgrid(np.arange(nx), np.arange(ny))
    res = np.zeros((nchannels, nx, ny))
    res[frame.T.ravel(), x.ravel(), y.ravel()] = 1
    return res

In [4]:
# convert to one-hot
# 32 x image
# for each pixel, have 1 value that is 1 (corresponding to that pixel's channel/class), and 31 values that are 0
frame_1hot = ...

In [5]:
cap = cv2.VideoCapture("/home/jupyter/.fastai/data/camvid/opticalFlow/train.mp4")

In [6]:
ret, frame1 = cap.read()

In [7]:
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY) 
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

In [8]:
type(prvs)

numpy.ndarray

### Use this method to visualize images

- Test out save to image portion of the method

In [9]:
def visualizeIT(ang, mag, hsv, VISUALIZE_FLOW):
        if VISUALIZE_FLOW:
            hsv[...,0] = ang*180/np.pi/2
            hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
            rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

            cv2.imshow('frame2',rgb)
            k = cv2.waitKey(30) & 0xff
        if k == 27:
            pass
        elif k == ord('s'):
            
            # Not sure if this saving method will work
            cv2.imwrite('opticalfb.png',frame2)
            cv2.imwrite('opticalhsv.png',rgb)

### Test on <10 frames

In [10]:
ret, frame2 = cap.read()
next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)


mag, ang = (flow[..., 0], flow[..., 1])
# mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

In [13]:
magAng = np.stack([mag, ang])
type(magAng)

numpy.ndarray

In [33]:
# Convert from torch tensor to numpy array 
magAngImage = torch.from_numpy(magAng)
angImage = torch.from_numpy(ang)
magImage = torch.from_numpy(mag)

#### Save to an image
Convert numpy array to torch Tensor

In [32]:
# does saving the cartesian coordinates as images preserve the cartesian coords? 
# Do I need to do anything else with the image / matrix before the next step?

save_image(magAngImage, 'MagAngImage33.png')
save_image(magImage, 'magImage33.png')
save_image(angImage, 'angImage33.png')

### Use this for mag or ang, but not for magAng

In [26]:
import matplotlib

matplotlib.image.imsave('MagOnly.png', mag)

## Try on ALL frames
- this loop works 

In [15]:
# True == visualize the motion
VISUALIZE_FLOW = False
count = 1

while(1):
    # compute optical flow for each frame
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Save mag, ang as information relevant to motion at each pixel 
#     mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    mag, ang = (flow[..., 0], flow[..., 1])

    # Combine mag, ang into one variable 
    magAng = np.stack([mag, ang]) # frame2.shape x 2
    magAngImage = torch.from_numpy(magAng)
    
    save_image(magAngImage, f'/home/jupyter/.fastai/data/camvid/opticalFlow/optFlow{count}.png')
#     cv2.imwrite(f'/home/jupyter/.fastai/data/camvid/opticalFlow/optFlow{count}.png', magAngImage)
    
#     save_image(magang, f'optFlow{count}.png')
    # save out frame2 motion
#     save_image(mag, ...) # how much motion?
#     save_image(ang, ...) # motion in what direction?


    # Set prvs to next image
    prvs = next
    count += 1
    
    # visualize the motion of frame2
#     visualizeIT(ang, mag, hsv, VISUALIZE_FLOW)

error: OpenCV(4.1.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [16]:
count

20398

### Pooling and One-Hot Encoding

In [ ]:
for frame in frames:
    # load frame_1hot for frame
    ...
    # load magang for frame
    ...
  
    # combine flow and segmentation information
    features = np.dot(frame_1hot, magang) # 32x2ximage
    
    # pool to downsample
    # use mean pooling over 3x3 grid on the image dimensions
    # 32x2x(3x3) vector of length 576
    features = pool(features)
    
    vector = np.ravel(features)
    

cap.release()
cv2.destroyAllWindows()

In [8]:
# mean pooling
from torch.nn.modules.pooling import AvgPool2d
# pool = AvgPool2d((img.shape[0]//3,img.shape[1]//3))
# ...
# for frame in frames:
# ...
# features = pool(features)